In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("./all.csv", index_col=0)

In [3]:
df.head()

,kadencja,etap,osoba,klub
1,1,start,Adam Glapiński,Klub Parlamentarny Porozumienia Centrum
2,1,end,Adam Glapiński,Klub Parlamentarny Porozumienia Centrum
3,1,start,Adam Halber,Polska Partia Przyjaciół Piwa
4,1,end,Adam Halber,Sojusz Lewicy Demokratycznej Klub Parlamentarny
5,1,start,Adam Lipiński,Klub Parlamentarny Porozumienia Centrum


In [8]:
df_kluby_slowniczek = pd.read_csv("./slownik_klubow.csv", index_col=0)

In [9]:
kluby_dict = {v['old']: v['klub'] for k, v in df_kluby_slowniczek.iterrows()}

In [12]:
df['klub'] = df['klub'].map(kluby_dict)

In [14]:
df.head()

,kadencja,etap,osoba,klub
1,1,start,Adam Glapiński,Porozumienie Centrum
2,1,end,Adam Glapiński,Porozumienie Centrum
3,1,start,Adam Halber,Polska Partia Przyjaciół Piwa
4,1,end,Adam Halber,Sojusz Lewicy Demokratycznej
5,1,start,Adam Lipiński,Porozumienie Centrum


In [25]:
# start i end wyglądają teraz OK
df[(df.etap=='end') & (df.kadencja==7)].klub.value_counts()

Platforma Obywatelska           197
Prawo i Sprawiedliwość          134
Polskie Stronnictwo Ludowe       38
Sojusz Lewicy Demokratycznej     35
Posłowie niezrzeszeni            25
Zjednoczona Prawica              16
Ruch Palikota                    11
Biało-Czerwoni                    4
dtype: int64

In [26]:
df["kadencja_ef"] = df["kadencja"] + (df["etap"] == "end")/2

In [51]:
przejscia = {}
for osoba, df_osoby in df.groupby('osoba'):
    df_osoby = df_osoby.sort(columns=["kadencja_ef"])
    for i in range(len(df_osoby) - 1):
        klub_przed = "{0} ({1:.1f})".format(df_osoby.iloc[i]["klub"], df_osoby.iloc[i]["kadencja_ef"])
        klub_po = "{0} ({1:.1f})".format(df_osoby.iloc[i + 1]["klub"], df_osoby.iloc[i + 1]["kadencja_ef"])
        if (klub_przed, klub_po) not in przejscia:
            przejscia[(klub_przed, klub_po)] = 1
        else:
            przejscia[(klub_przed, klub_po)] += 1

In [53]:
df_przejscia = pd.DataFrame([{'source': k1, 'target': k2, 'value': v}
                             for (k1, k2), v in przejscia.items()])

In [54]:
df_przejscia.head()

,source,target,value
0,Sojusz Lewicy Demokratycznej (7.0),Posłowie niezrzeszeni (7.5),1
1,Posłowie niezrzeszeni (2.5),Sojusz Lewicy Demokratycznej (3.0),1
2,Ruch dla Rzeczypospolitej (1.5),Ruch Odbudowy Polski (3.0),2
3,Platforma Obywatelska (4.0),Prawo i Sprawiedliwość (4.5),1
4,Unia Wolności (3.0),Unia Wolności (3.5),47


In [55]:
df_przejscia.to_csv("przejscia.csv", index=False)